# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from matplotlib import colormaps
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,25.32,93.0,100.0,11.50,CA,2024-11-07 03:22:36
1,1,mujiayingzi,42.1167,118.7833,46.80,36.0,2.0,5.77,CN,2024-11-07 03:22:42
2,2,newman,37.3138,-121.0208,54.05,57.0,1.0,1.99,US,2024-11-07 03:22:42
3,3,puspokladany,47.3167,21.1167,34.97,61.0,26.0,4.47,HU,2024-11-07 03:22:42
4,4,bilibino,68.0546,166.4372,-8.34,100.0,100.0,1.90,RU,2024-11-07 03:22:43


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    framewise=True,
    height=700,
    width=900,
    padding=0.05,
    size="Humidity",
    color="Humidity",
    cmap="RdYlBu",
    colorbar=True,
    tiles="OSM",
    hover_cols=["City", "Country", "Humidity", "Max Temp", "Wind Speed", "Cloudiness"]
)
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Max Temp,Wind Speed,Cloudiness)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,25.32,93.0,100.0,11.50,CA,2024-11-07 03:22:36
1,1,mujiayingzi,42.1167,118.7833,46.80,36.0,2.0,5.77,CN,2024-11-07 03:22:42
2,2,newman,37.3138,-121.0208,54.05,57.0,1.0,1.99,US,2024-11-07 03:22:42
3,3,puspokladany,47.3167,21.1167,34.97,61.0,26.0,4.47,HU,2024-11-07 03:22:42
4,4,bilibino,68.0546,166.4372,-8.34,100.0,100.0,1.90,RU,2024-11-07 03:22:43


In [5]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 40
max_temp = 80 
max_humidity = 60 
max_clouds = 60 
max_wind = 10

# Drop any rows with null values
# I completed this in the WeatherPy assignment to not interfere with graphing and regression review.
cities_filtered = city_data_df[
    (city_data_df["Max Temp"] >= min_temp) &  
    (city_data_df["Max Temp"] <= max_temp) &
    (city_data_df["Humidity"] <= max_humidity) &
    (city_data_df["Cloudiness"] <= max_clouds) &
    (city_data_df["Wind Speed"] <= max_wind)
]


# Display sample data
cities_filtered.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,mujiayingzi,42.1167,118.7833,46.80,36.0,2.0,5.77,CN,2024-11-07 03:22:42
2,2,newman,37.3138,-121.0208,54.05,57.0,1.0,1.99,US,2024-11-07 03:22:42
8,8,alibag,18.6411,72.8792,79.30,50.0,5.0,6.60,IN,2024-11-07 03:22:43
18,18,al kharijah,25.4514,30.5464,61.61,53.0,0.0,6.62,EG,2024-11-07 03:22:44
25,26,blackmans bay,-43.0167,147.3167,70.52,42.0,28.0,1.01,AU,2024-11-07 03:22:46


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_filtered.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(10,"Hotel Name", "")

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,mujiayingzi,42.1167,118.7833,46.80,36.0,2.0,5.77,CN,2024-11-07 03:22:42,
2,2,newman,37.3138,-121.0208,54.05,57.0,1.0,1.99,US,2024-11-07 03:22:42,
8,8,alibag,18.6411,72.8792,79.30,50.0,5.0,6.60,IN,2024-11-07 03:22:43,
18,18,al kharijah,25.4514,30.5464,61.61,53.0,0.0,6.62,EG,2024-11-07 03:22:44,
25,26,blackmans bay,-43.0167,147.3167,70.52,42.0,28.0,1.01,AU,2024-11-07 03:22:46,


In [7]:
hotel_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Hotel Name'],
      dtype='object')

In [10]:
# Drop & Reorder columns to align with example
hotel_df = hotel_df.drop(columns=["Max Temp", "Cloudiness", "Wind Speed", "Date"])
columns_order = ["City_ID", "City", "Country", "Humidity", "Lat", "Lng", "Hotel Name"]
hotel_df= hotel_df[columns_order]

hotel_df.head()

,City_ID,City,Country,Humidity,Lat,Lng,Hotel Name
1,1,mujiayingzi,CN,36.0,42.1167,118.7833,
2,2,newman,US,57.0,37.3138,-121.0208,
8,8,alibag,IN,50.0,18.6411,72.8792,
18,18,al kharijah,EG,53.0,25.4514,30.5464,
25,26,blackmans bay,AU,42.0,-43.0167,147.3167,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Base URL for the Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

def search_hotels(lat, lng, api_key):
    radius = 10000  # Radius in meters
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "limit": 1,  # Set the limit to 1 to get the closest hotel
        "apiKey": geoapify_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["features"]:
            try:
                return data["features"][0]["properties"]["name"]
            except KeyError:
                return "No hotel name found"
        else:
            return "No hotel found"
    else:
        return f"Error: {response.status_code}"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through hotel_df
for index, row in hotel_df.iterrows():
    radius = 10000
    hotel_name = search_hotels(row["Lat"], row["Lng"], geoapify_key)
    hotel_df.at[index, "Hotel Name"] = hotel_name


    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{row["Lng"]},{row["Lat"]},{radius}",
        "limit": 1,  # Set the limit to 1 to get the closest hotel
        "apiKey": geoapify_key,
        "bias": f"proximity:{row["Lng"]},{row["Lat"]}"
    }

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")

# Display updated dataFrame with hotels.
hotel_df.head()

Starting hotel search
mujiayingzi - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
alibag - nearest hotel: Chirag executive
al kharijah - nearest hotel: Kharga Hotel
blackmans bay - nearest hotel: Villa Howden
sur - nearest hotel: Sur Hotel
katghora - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
tiznit - nearest hotel: No hotel found
mudhol - nearest hotel: No hotel found
viedma - nearest hotel: Níjar
recani - nearest hotel: Hotel Merzouga
atar - nearest hotel: Residence Egueil
murzuq - nearest hotel: فندق باريس - مرزق
al mindak - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
las heras - nearest hotel: Posadas del Sur
rikaze - nearest hotel: Gesar Hotel
kant - nearest hotel: No hotel found
jumla - nearest hotel: Amar Sandesh Guesthouse
tan-tan - nearest hotel: Sable d´or
el bayadh - nearest hotel: No hotel found
korba - nearest hotel: No hotel found
darya khan - nearest hotel: No hotel found
sabha - nearest h

,City_ID,City,Country,Humidity,Lat,Lng,Hotel Name
1,1,mujiayingzi,CN,36.0,42.1167,118.7833,No hotel found
2,2,newman,US,57.0,37.3138,-121.0208,No hotel found
8,8,alibag,IN,50.0,18.6411,72.8792,Chirag executive
18,18,al kharijah,EG,53.0,25.4514,30.5464,Kharga Hotel
25,26,blackmans bay,AU,42.0,-43.0167,147.3167,Villa Howden


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    framewise=True,
    height=700,
    width=900,
    padding=0.05,
    size="Humidity",
    color="Humidity",
    cmap="RdYlBu",
    colorbar=True,
    tiles="OSM",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)